In [2]:
import numpy as np
import pandas as pd

from datetime import datetime

In [5]:
# city: input(Where are you going?) ---> Drop down list (Köln, München)
city = 'Köln'
# weekday: input(When are you going?) ---> Calendar menu (no more than 8 days (?))
date = datetime(2022, 6, 8, 5)
weekday = date.weekday()
month = date.month
time_of_day = date.time()
# print(weekday, month, time_of_day)




2 6 05:00:00


In [ ]:
X = data[['city', 'temp', 'prcp', 'wspd', 'wpgt', 'snow', 'coco', 'weekday', 'month', 'time_of_day']]